<a href="https://colab.research.google.com/github/niemand-01/ML-Demo/blob/master/Hyperparameter_tuning_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow import keras
import IPython
from google.colab.patches import cv2_imshow

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


In [8]:
(x_train,y_train),(x_test,y_test) = keras.datasets.fashion_mnist.load_data()

img_train = x_train.astype('float32')/255.0
img_test = x_test.astype('float32')/255.0

cv2_imshow(img_train[0])

In [10]:
# model
def model(hp):
  hp_units = hp.Int('units',min_value=32,max_value=512,step = 32)
  model = keras.Sequential([
          keras.layers.Flatten(input_shape=(28,28)),
          keras.layers.Dense(units=hp_units,activation='relu'),
          keras.layers.Dense(10)
  ])

  hp_learning_rate = hp.Choice('learning_rate',values = [1e-2,1e-3,1e-4])

  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['accuracy']
  )

  return model

In [13]:
# hyperband instantiate
tuner = kt.Hyperband(model,
            objective = 'val_accuracy',
            max_epochs =10,
            factor =3,
            directory = 'logs/hptuning',
            project_name='intro_to_kt')

# extend Callback class
# clear log
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args,**kwargs):
    IPython.display.clear_output(wait = True)

tuner.search(img_train,y_train,epochs = 10,validation_data = (img_test,y_test), callbacks = [ClearTrainingOutput()])

# print best hp
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

INFO:tensorflow:Reloading Oracle from existing project logs/hptuning/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from logs/hptuning/intro_to_kt/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [15]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, y_train, epochs = 10, validation_data = (img_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4728 - accuracy: 0.8337 - val_loss: 0.4073 - val_accuracy: 0.8541
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3621 - accuracy: 0.8681 - val_loss: 0.4059 - val_accuracy: 0.8517
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3244 - accuracy: 0.8804 - val_loss: 0.3637 - val_accuracy: 0.8724
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3013 - accuracy: 0.8891 - val_loss: 0.3546 - val_accuracy: 0.8719
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2824 - accuracy: 0.8960 - val_loss: 0.3693 - val_accuracy: 0.8626
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2669 - accuracy: 0.8985 - val_loss: 0.3336 - val_accuracy: 0.8823
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2547 - accuracy: 0.9042 - val_loss: 0.3297 - val_accuracy: